# Preeliminary: Import and loading data

In [ ]:
import os
HOME = os.getcwd()
from pathlib import Path
print(HOME)

HOME_PARENT = Path(HOME).parent
import sys
sys.path.append(str(HOME_PARENT))
sys.path.append(os.path.join(HOME_PARENT, 'scripts'))
print(sys.path)

In [ ]:
import pandas as pd
CA_PATH = os.path.join(HOME_PARENT, 'data', 'casualities.csv')
ACC_PATH = os.path.join(HOME_PARENT, 'data', 'accidents.csv')
VE_PATH = os.path.join(HOME_PARENT, 'data', 'vehicles.csv')

In [ ]:
# let's load the data as needed
ACC = pd.read_csv(ACC_PATH)


Considering the number data suggested by the 3 tables, we can see that the best way to combine the data available is to predict the casuality severity.

# The data: a dive into the data's documentation 
## Accidents

Going through the explanations of the fields, it is possible to reduce the number of features as some of them do not introduce any additional value for our predictive task.  
The following will be dropped:  
1. Location Easting OSGR (Null if not known)
2. Location Northing OSGR (Null if not known)    
These values represent the location of the accident with respect to a local geospatial system
3. The police attendance
4. Longitude and Latitude might be dropped as the data is already clustered into different districts
5. Accident Severity: this value is practically equivalent to the target: the severity of casualities
6. Police: The police's intervention takes place generally after the accident. Such intervention could not possible affect the accident's severity and the casualities' seriousness 

Additional remarks:
* The most seemingly important features are:
    1. 1st /2nd  Road Class / if it reflects quality
    2. weather conditions / Light Conditions
    3. Pedestrian Crossing Human control: we don't expect many accidents in conjuctions controlled by police officer: HOWEVER IT MIGHT HAVE SOME OVERLAPPING WITH CONJUNCTION CONTROL
    4. Urban / Rural area: Rural area are more likely to have more fatal accidents: more serious casualities
    5. SPEED LIMIT
These observations are to confirmed to denied through the EDA.
* geospatial information should be processed further or dropped
* The temporal data might be reduced to either the month or the year values if needed

## The vehicles


* The data provides a detailed description of the vehicle
* The fields most likely should be combined into a fewer but more general representations
* Certain fields might be dropped:
    1. Vehicle Location: can be deduced to a certain extent by the type of the road / location the accident took place
    2. Vehicle Maneouver is to be dropped
    3. There are two Hit Object features that can be merged into one
    4. The IMD level as well as the home area of the driver do not seem to have direct relation with the seriousness of the casuality
    5. any information about the driver can be found in the casuality table, so it should be dropped from the vehicle table 


# The data: a dive into the code

## The accidents table

In [ ]:
# first let's start with removing the unncessary data
ACC.columns

In [ ]:
acc_drop_cols = ['Location_Easting_OSGR', 'Location_Northing_OSGR','Police_Force', 'Did_Police_Officer_Attend_Scene_of_Accident', 'LSOA_of_Accident_Location', 'Number_of_Casualties']
ACC.drop(columns=acc_drop_cols, inplace=True)

In [ ]:
ACC.columns

In [ ]:
# let's provide better names (and mainly shorter) for the featrures
new_acc_cols = {'Longitude': 'lng', 'Latitude': 'lat', 'Accident_Severity': 'y', 'Number_of_Vehicles': 'n_vehs', 'Local_Authority_(District)': 'district',
                'Local_Authority_(Highway)': 'highway', '1st_Road_Class': 'road_c1', '1st_Road_Number': 'road_n1', 'Road_Type':'road_type'
                , 'Light_Conditions': 'light', 'Weather_Conditions': 'weather', 'Road_Surface_Conditions': 'road_surface', 'Urban_or_Rural_Area': 'area_type'
                , "Junction_Detail": "junc_detail", "Junction_Control": "junc_control", "2nd_Road_Class": "road_c2", "2nd_Road_Number": "road_n2", 
                "Pedestrian_Crossing-Human_Control": "cross_control", "Pedestrian_Crossing-Physical_Facilities": "cross_facilities", 
                "Special_Conditions_at_Site": "special_conds", "Carriageway_Hazards": "hazards"}

from df_operations import new_col_names, to_columns
ACC = new_col_names(new_acc_cols, ACC)
# make sure to convert them to lowercase
ACC = to_columns(ACC, lambda c: c.lower().strip()) 

### ACC: Explatory Data Analysis

In [ ]:
# first let's split the data
from df_operations import get_col_types, draw_unique_data_table, draw_missing_data_table
num_cols, cat_cols = get_col_types(ACC)

In [ ]:
draw_unique_data_table(ACC)
# let's start with a small number of 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.clf()
acc_cat = ACC.loc[:, cat_cols]

small_cats = [c for c in  cat_cols if len(acc_cat[c].value_counts()) <= 10]
print(len(small_cats))


In [ ]:
figure, axes = plt.subplots(4, 4,figsize=(20, 16), sharex=False, sharey=False)
for i, f in enumerate(small_cats):  
    chart = sns.histplot(ax=axes[i // 4, i % 4], data=acc_cat, x=f, stat='percent')
    # chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
plt.show()

In [ ]:
# the following procedure will be used:
# if a certain value of a categorical variable does not correlate with fatal accidents: (a mere 2% of the total accidents), then this feature will either be dropped, or 
# the values will be rearranged
import numpy as np

def cat_feat_distribution(df: pd.DataFrame, feature: str, n_unique_as_discrete=20):
    assert feature in df.columns
    # make sure the indeed categorical
    assert feature not in set(list(df.select_dtypes(np.number).columns)) or len(df[feature].value_counts()) <= n_unique_as_discrete 

    return (df[feature].value_counts() / len(df))

for c in small_cats:
    print(cat_feat_distribution(acc_cat, c))

In [ ]:
len(acc_cat)

In [ ]:
import visual as v
# now it is interesting to understand the conditional probabitili

# for c in small_cats:
#     figure, axes = plt.subplots(1, 2, figsize=(18, 6), sharex=False, sharey=False)
#     # first display distribution
#     distribution = sns.histplot(ax=axes[0], data=acc_cat, x=c, stat='percent')
#     # second display the condition distribution of the categorical feature over the target variable
#     con_dis = v.visualize_cond_prob(acc_cat, target='y', hue=c, show=False)
#     axes[1] = con_dis

#     # con_dis_2 = v.visualize_cond_prob(acc_cat, target=c, hue='y', show=False)
#     # axes[2] = con_dis_2

#     plt.show()



In [ ]:
# the day of the week does not seem to affect the seriousness of the casuality
ACC.drop(columns=['day_of_week', 'y', 'n_vehs'], inplace=True)

In [ ]:
# let's make sure to convert the date to a datetime4
# first convert all dates to the same format

from  datetime import datetime
from dateutil.parser import parse

def uniform_date_format(date_text:str):
    try:
        return datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        pass
    # first parse the date as it is
    dt = parse(date_text)
    return dt.strftime('%d/%m/%Y')

# first make sure to unify the date's format
ACC['date'] = ACC['date'].apply(uniform_date_format)
# convert all the strings to date time objects
ACC['date'] = pd.to_datetime(ACC['date'], format='%d/%m/%Y')

In [ ]:
ACC.dtypes

In [ ]:
# one final important step to do is to convert the textual indices
ACC.reset_index(inplace=True)
# create a map to map each old accident_index to the new index
old_to_index_map = dict([(old, new) for old, new in zip(ACC['accident_index'].tolist(), ACC['index'].tolist())])
ACC.drop(columns='accident_index', inplace=True)
ACC = ACC.rename(columns={"index": "accident_index"})

In [ ]:
ACC.head()

In [ ]:
# save the resulting accidents table
processed_data_dir = os.path.join(HOME_PARENT, 'data', 'preprocessed_data')
os.makedirs(processed_data_dir, exist_ok=True)
ACC.to_csv(os.path.join(processed_data_dir, 'accidents_v1.csv'), index=False)

## The Vehicles Data

In [ ]:
VE = pd.read_csv(VE_PATH)
VE = to_columns(VE, lambda x: x.lower().strip())
print(VE.columns)
COLUMNS_TO_REMOVE = ['vehicle_manoeuvre', 'vehicle_location-restricted_lane', 'junction_location', 'was_vehicle_left_hand_drive?', 
                     'journey_purpose_of_driver', 'sex_of_driver', 'age_of_driver','age_band_of_driver', 'propulsion_code',
                     'driver_imd_decile', 'driver_home_area_type']

VE.drop(columns=COLUMNS_TO_REMOVE, inplace=True)
VE.head()

In [ ]:
VE.columns
# renaming the columns
NEW_VEH_NAMES = {"accident_index": "acc_index", "vehicle_reference": "veh_ref", "vehicle_type": "veh_type", "towing_and_articulation": "towing",
                 "skidding_and_overturning": "reversed", "hit_object_in_carriageway": "object_in", "vehicle_leaving_carriageway": "veh_left", 
                 "hit_object_off_carriageway": "object_out", '1st_point_of_impact':"impact","engine_capacity_(cc)": "cc", "age_of_vehicle": "veh_age"}

VE = new_col_names(NEW_VEH_NAMES, VE) 

In [ ]:
VE['acc_index'] = VE['acc_index'].apply(lambda x: old_to_index_map[x]) 
assert set(VE['acc_index']).issubset(ACC['accident_index'])
assert len(VE['acc_index'].value_counts()) == len(ACC)


THE vehicle table does not have a unique reference for the car. The primary key is veh_ref + acc_index. To load this data to a database, the veh_ref should be modified to uniquely reprsent the car: a simple merge of veh_ref and accident_index would do the trick.

In [ ]:
print(max(VE['veh_ref']))
print(min(VE['veh_ref']))

In [ ]:
def fix_veh_ref(row):
    row['veh_ref'] = row['acc_index'] * 91 + row['veh_ref'] 
    return row

VE = VE.apply(fix_veh_ref, axis=1)
assert all(VE['veh_ref'].value_counts() == 1)

In [ ]:
VE.to_csv(os.path.join(processed_data_dir, 'vehicles_v1.csv'), index=False)

## The casualities Data

In [ ]:
CA = pd.read_csv(CA_PATH)
CA.columns = [c.lower() for c in CA.columns]
print(CA.columns)
# drop the last 6 columns
CA.drop(columns=list(CA.columns)[-6:] + ['casualty_reference'], inplace=True)
CA.columns

In [ ]:
# let's fix the naming here
CA_NEW_NAMES = {"accident_index": "acc_index", "vehicle_reference": "veh_ref", 'casualty_class': "cas_type","sex_of_casualty": "cas_sex", 
                'age_of_casualty': "cas_age", 'age_band_of_casualty': "cas_age_band", 'casualty_severity': "cas_y", 'pedestrian_location': "ped_loc"}

CA = new_col_names(CA_NEW_NAMES, CA)
CA.columns

In [ ]:
# as it does not seem possible to make a primary key for each casualty out of the provided data
# we can create a simple serieal key by resetting the index
CA.reset_index(inplace=True)
CA.head()

In [ ]:
CA['acc_index'] = CA['acc_index'].apply(lambda x: old_to_index_map[x]) 
assert set(CA['acc_index']) == set(ACC['accident_index'])
assert len(CA['acc_index'].value_counts()) == len(ACC)

In [ ]:
# the same modification applied on the vehicle file should be applied to that of casualties
CA = CA.apply(fix_veh_ref, axis=1)
assert set(CA['veh_ref']).issubset(set(VE['veh_ref']))

In [ ]:
CA.to_csv(os.path.join(processed_data_dir, 'casualties_v1.csv'), index=False)

# Namesless Section

In [1]:
import pandas as pd
import os
from pathlib import Path

print
processed_data_dir = os.path.join(Path(os.getcwd()).parent, 'data', 'preprocessed_data')
# let's get this shit done by today!!
ACC = pd.read_csv(os.path.join(processed_data_dir, 'accidents_v1.csv'))
CA = pd.read_csv(os.path.join(processed_data_dir, 'casualties_v1.csv'))
VEH = pd.read_csv(os.path.join(processed_data_dir, 'vehicles_v1.csv'))

print(ACC.columns)
print(CA.columns)
print(VEH.columns)


Index(['accident_index', 'lng', 'lat', 'date', 'district', 'road_c1',
       'road_n1', 'road_type', 'speed_limit', 'junc_detail', 'junc_control',
       'road_c2', 'road_n2', 'cross_control', 'cross_facilities', 'light',
       'weather', 'road_surface', 'special_conds', 'hazards', 'area_type'],
      dtype='object')
Index(['index', 'acc_index', 'veh_ref', 'cas_type', 'cas_sex', 'cas_age',
       'cas_age_band', 'cas_y', 'ped_loc'],
      dtype='object')
Index(['acc_index', 'veh_ref', 'veh_type', 'towing', 'reversed', 'object_in',
       'veh_left', 'object_out', 'impact', 'cc', 'veh_age'],
      dtype='object')


In [2]:
# let's check the types 
print(ACC.dtypes, end='\n' * 2)
print(CA.dtypes, end='\n' * 2)
print(VEH.dtypes, end='\n' * 2)

accident_index        int64
lng                 float64
lat                 float64
date                 object
district              int64
road_c1               int64
road_n1               int64
road_type             int64
speed_limit           int64
junc_detail           int64
junc_control          int64
road_c2               int64
road_n2               int64
cross_control         int64
cross_facilities      int64
light                 int64
weather               int64
road_surface          int64
special_conds         int64
hazards               int64
area_type             int64
dtype: object

index           int64
acc_index       int64
veh_ref         int64
cas_type        int64
cas_sex         int64
cas_age         int64
cas_age_band    int64
cas_y           int64
ped_loc         int64
dtype: object

acc_index     int64
veh_ref       int64
veh_type      int64
towing        int64
reversed      int64
object_in     int64
veh_left      int64
object_out    int64
impact        int64
cc  

In [ ]:
ACC.drop(columns=['highway', 'time'], inplace=True)
ACC.to_csv(os.path.join(processed_data_dir, 'accidents_v1.csv'), index=False) 
ACC.dtypes